In [2]:
# Import dependencies
import pandas as pd

In [107]:
df = pd.read_csv('data_prepro/allStats.csv', index_col=0)

In [108]:
len(df)

68749

In [109]:
# Clean defense names in historical data
# Create function to check if last three characters are uppercase. Strip only the uppercase
def cleanDefNames(text):
    if text[-3:].isupper():
        text = text[:-3]
    else:
        text = text[:-2]
    return(text)
# Apply the function to the defenses
df.loc[df.pos=='DF', 'player'] = df.loc[df.pos=='DF', 'player'].apply(cleanDefNames)

# Rename past teams to their most recent name to account for name changes or city moves
df.team.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.player.replace(to_replace=['St. Louis Rams'], value='Los Angeles Rams', inplace=True)
df.team.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.player.replace(to_replace=['Oakland Raiders'], value='Las Vegas Raiders', inplace=True)
df.team.replace(to_replace=['NE'], value='NEP', inplace=True)
df.team.replace(to_replace=['SD'], value='LAC', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)
df.team.replace(to_replace=['NO'], value='NOS', inplace=True)
df.team.replace(to_replace=['GB'], value='GBP', inplace=True)
df.team.replace(to_replace=['KC'], value='KCC', inplace=True)
df.team.replace(to_replace=['SF'], value='SFO', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)

df.opponent.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.opponent.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.opponent.replace(to_replace=['NE'], value='NEP', inplace=True)
df.opponent.replace(to_replace=['SD'], value='LAC', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)
df.opponent.replace(to_replace=['NO'], value='NOS', inplace=True)
df.opponent.replace(to_replace=['GB'], value='GBP', inplace=True)
df.opponent.replace(to_replace=['KC'], value='KCC', inplace=True)
df.opponent.replace(to_replace=['SF'], value='SFO', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)

In [110]:
len(df)

68749

In [111]:
# Set NaN values to zero
df.fillna(0, inplace=True)

# Handle duplicates
dupList = df.loc[df['pos']!='DF'].groupby(['player', 'season', 'week'])['player'].count().sort_values(ascending=False)
dupList = dupList.loc[dupList>1].to_frame().index.get_level_values(0).drop_duplicates()
# Handle 2022 duplicates by appending position to player name
for p in dupList:
    df.loc[df.player==p, 'player'] = df.loc[df.player==p, 'player'] + df.loc[df.player==p, 'pos']


In [112]:
len(df)

68749

In [114]:
# Create summary df for current season prior to current week
currentSeason_df = df.copy()
# Create empty list
data=[]

# Loop through df
for i in range(len(currentSeason_df)):
    season = currentSeason_df.iloc[i]['season']
    player = currentSeason_df.iloc[i]['player']
    week = currentSeason_df.iloc[i]['week']
# filter for player where week < current week
    prevWeeks = currentSeason_df.loc[(currentSeason_df['season']==season) & (currentSeason_df['player']==player) & (currentSeason_df['week']<week)]
    ind = [season, week, player]
    prevComplete = []
    if len(prevWeeks) > 0:
     # take average
        guts = prevWeeks[['passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
        'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
        'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
        'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
        'defYdsAgainst']].mean().to_list()
        
    else:
        guts = [0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0]
    ind.extend(guts)
    # # groupby and count games played
    gaPlayed = [len(prevWeeks)]
    ind.extend(gaPlayed)
    data.append(ind)
# Errors if no weeks < week
seasonCurrent = pd.DataFrame(data, columns=['season', 'week', 'player',
       'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
       'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
       'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
       'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
       'defYdsAgainst', 'gamesPlayed'])

# Rename all columns in seasonCurrent
colList = [(x + "_curr") for x in list(seasonCurrent.columns)]
seasonCurrent.columns = colList
seasonCurrent = seasonCurrent.rename(columns={'season_curr':'season',
 'week_curr':'week',
 'player_curr':'player'})


In [119]:
len(seasonCurrent)

68749

In [115]:

#Create summary dfs for each season
seasonsPrior1_df = df.copy()
seasonsPrior1_df['season'] = seasonsPrior1_df['season'] + 1
seasonsPrior1 = seasonsPrior1_df.groupby(['season', 'player']).mean()
seasonsPrior1['gamesPlayed'] = seasonsPrior1_df.groupby(['season', 'player'])['week'].count()
seasonsPrior1.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior1.pop('gamesPlayed')
seasonsPrior1.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior1
colList = [(x + "_prior1") for x in list(seasonsPrior1.columns)]
seasonsPrior1.columns = colList
seasonsPrior1

# Create summary dfs for combined two seasons before
seasonsPrior2_df = df.copy()
seasonsPrior2_df['season'] = seasonsPrior2_df['season'] + 2
seasonsPrior2_df = pd.concat([seasonsPrior1_df, seasonsPrior2_df], axis=0, ignore_index=True)
seasonsPrior2 = seasonsPrior2_df.groupby(['season', 'player']).mean()
seasonsPrior2['gamesPlayed'] = seasonsPrior2_df.groupby(['season', 'player'])['week'].count()
seasonsPrior2.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior2.pop('gamesPlayed')
seasonsPrior2.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior2
colList = [(x + "_prior2") for x in list(seasonsPrior2.columns)]
seasonsPrior2.columns = colList
seasonsPrior2

gamesPlayed_prior2  passA_prior2  passC_prior2  \
season player                                                            
2011.0 Aaron Brown                       3      0.000000      0.000000   
       Aaron Hernandez                  13      0.000000      0.000000   
       Aaron Pettrey                     2      0.000000      0.000000   
       Aaron Rodgers                    15     31.666667     20.800000   
       Adam Vinatieri                   16      0.000000      0.000000   
...                                    ...           ...           ...   
2023.0 Zach Pascal                      14      0.000000      0.000000   
       Zach Wilson                      12     30.250000     17.166667   
       Zack Moss                        12      0.000000      0.000000   
       Zane Gonzalez                    12      0.000000      0.000000   
       Zay Jones                        14      0.000000      0.000000   

                        passY_prior2  passT_prior2  passI_prior2  \
season player                                                      
2011.0 Aaron Brown          0.000000      0.000000      0.000000   
       Aaron Hernandez      0.000000      0.000000      0.000000   
       Aaron Pettrey        0.000000      0.000000      0.000000   
       Aaron Rodgers      261.466667      1.866667      0.733333   
       Adam Vinatieri       0.000000      0.000000      0.000000   
...                              ...           ...           ...   
2023.0 Zach Pascal          0.000000      0.000000      0.000000   
       Zach Wilson        187.250000      0.666667      0.916667   
       Zack Moss            0.000000      0.000000      0.000000   
       Zane Gonzalez        0.000000      0.000000      0.000000   
       Zay Jones            0.000000      0.000000      0.000000   

                        pass2_prior2  rushA_prior2  rushY_prior2  \
season player                                                      
2011.0 Aaron Brown          0.000000      5.666667     19.333333   
       Aaron Hernandez      0.000000      0.230769      3.615385   
       Aaron Pettrey        0.000000      0.000000      0.000000   
       Aaron Rodgers        0.000000      4.266667     23.733333   
       Adam Vinatieri       0.000000      0.000000      0.000000   
...                              ...           ...           ...   
2023.0 Zach Pascal          0.000000      0.142857      1.500000   
       Zach Wilson          0.083333      2.250000     13.416667   
       Zack Moss            0.000000      7.583333     28.083333   
       Zane Gonzalez        0.000000      0.000000      0.000000   
       Zay Jones            0.000000      0.142857      0.214286   

                        rushT_prior2  ...  defSack_prior2  defI_prior2  \
season player                         ...                                
2011.0 Aaron Brown          0.000000  ...             0.0          0.0   
       Aaron Hernandez      0.000000  ...             0.0          0.0   
       Aaron Pettrey        0.000000  ...             0.0          0.0   
       Aaron Rodgers        0.266667  ...             0.0          0.0   
       Adam Vinatieri       0.000000  ...             0.0          0.0   
...                              ...  ...             ...          ...   
2023.0 Zach Pascal          0.000000  ...             0.0          0.0   
       Zach Wilson          0.333333  ...             0.0          0.0   
       Zack Moss            0.333333  ...             0.0          0.0   
       Zane Gonzalez        0.000000  ...             0.0          0.0   
       Zay Jones            0.000000  ...             0.0          0.0   

                        defSaf_prior2  defFum_prior2  defBlk_prior2  \
season player                                                         
2011.0 Aaron Brown                0.0            0.0            0.0   
       Aaron Hernandez            0.0            0.0            0.0   
       Aaron Pettrey              0.0       

In [120]:
print(len(seasonsPrior1))
print(len(seasonsPrior2))

7432
9796


In [155]:
# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = df.copy()

# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = final.merge(
    seasonCurrent, how='left', on=['season', 'week', 'player']
    ).merge(
        seasonsPrior1, how='left', on=['season', 'player']
        ).merge(
            seasonsPrior2, how='left', on=['season', 'player']
            )
# Remove rows where we do not have two full years prior data
final = final.loc[final['season']>2011]
# fill NA values for players who do not have two years prior data
final.fillna(0, inplace=True)

In [156]:
print(len(final))
print(len(final.groupby(['player', 'season'])))


57483
6242


In [157]:
# select only defenses
allDef = final.loc[final['pos']=='DF']
# allDef.drop(columns=['passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
#        'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
#        'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
#        'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
#        'defYdsAgainst'], inplace=True)
# Drop non defense related cols
# allDef.drop(columns=['player', 'pos', 'posRank', 'opponent',
#        'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr',
#        'pass2_curr', 'rushA_curr', 'rushY_curr', 'rushT_curr', 'rush2_curr',
#        'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr',
#        'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 'FG50_curr',
#        'gamesPlayed_curr', 'gamesPlayed_prior1', 'passA_prior1',
#        'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1',
#        'pass2_prior1', 'rushA_prior1', 'rushY_prior1', 'rushT_prior1',
#        'rush2_prior1', 'recC_prior1', 'recY_prior1', 'recT_prior1',
#        'rec2_prior1', 'fum_prior1', 'XPA_prior1', 'XPM_prior1', 'FGA_prior1',
#        'FGM_prior1', 'FG50_prior1', 'gamesPlayed_prior2',
#        'passA_prior2', 'passC_prior2', 'passY_prior2', 'passT_prior2',
#        'passI_prior2', 'pass2_prior2', 'rushA_prior2', 'rushY_prior2',
#        'rushT_prior2', 'rush2_prior2', 'recC_prior2', 'recY_prior2',
#        'recT_prior2', 'rec2_prior2', 'fum_prior2', 'XPA_prior2', 'XPM_prior2',
#        'FGA_prior2', 'FGM_prior2', 'FG50_prior2'], inplace=True)

# Rename all columns in allDef
colList = [(x + "_opp") for x in list(allDef.columns)]
allDef.columns = colList
allDef = allDef.rename(columns={
       'season_opp':'season',
       'week_opp':'week',
       'team_opp':'team',
       'player_opp':'player'})

allDef = allDef[['season', 'week', 'team', 
       'defSack_curr_opp', 'defI_curr_opp',
       'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp',
       'defT_curr_opp', 'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp',
       'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp',
       'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp',
       'defFum_prior1_opp', 'defBlk_prior1_opp', 'defT_prior1_opp',
       'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp',
       'defRushYAgainst_prior1_opp', 'defYdsAgainst_prior1_opp',]]

# Connect opponents to defenses
final = final.merge(allDef, how='left', left_on=['season', 'week', 'opponent'], right_on=['season', 'week', 'team'],)
final.drop(columns=['team_y'], inplace=True)
final.rename(columns={'team_x':'team'}, inplace=True)

In [158]:
print(len(final))
print(len(final.groupby(['player', 'season'])))


57483
6242


In [159]:
# Read in ages
xl2 = pd.read_csv('data_cleaned/modelAges.csv', index_col=0)
xl2 = xl2.drop(columns=['week'])
xl2 = xl2.rename(columns={'Age':'age'})

# Change duplicate players' names to name + position
for p in dupList:
    xl2.loc[xl2.player==p, 'player'] = xl2.loc[xl2.player==p, 'player'] + xl2.loc[xl2.player==p, 'pos']

In [160]:
# merge ages into final df
final = final.merge(xl2, how='left', on=['season', 'team', 'player', 'pos'])
ageCol = final.pop('age')
final.insert(4, 'age', ageCol)

# Change defense ages to 0
final.loc[final['pos']=='DF', 'age'] = 0

In [161]:
print(len(final))
print(len(final.groupby(['player', 'season'])))
print(len(xl2.groupby(['player', 'season'])))

57483
6242
6160


In [162]:
xl2.columns

Index(['player', 'season', 'team', 'pos', 'age'], dtype='object')

In [164]:
final.loc[final['age'].isna()].player.unique()

array(['Alex SmithQB', 'E.J. Manuel', 'Phillip Walker', 'David JohnsonRB',
       'Greg Howell', 'Kerrith Whyte', 'Steve SmithWR',
       'Emmanuel Sanders', 'Lester Jean', 'Edmond Gates', 'Joshua Cribbs',
       'Joseph Morgan', 'Devon Wylie', 'Devin Hester', 'Greg Jenkins',
       'Walter Powell', 'Isaiah Burse', 'Matt Slater', 'Jesus Wilson',
       'J.J. Jones', 'Riley McCarron', "D'haquille Williams",
       'John Franklin III', 'Travis Benjamin', 'Alex SmithTE',
       'Benjamin Watson', 'Timothy Wright', 'Harold Hoskins',
       'Henry Krieger Coble'], dtype=object)

In [165]:
final = final.dropna()
final.describe()

,season,week,age,passA,passC,passY,passT,passI,pass2,rushA,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
count,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000,...,56768.000000,56768.000000,56768.000000,56768.000000,56768.00000,56768.000000,56768.000000,56768.000000,56768.000000,56768.000000
mean,2016.526652,9.041661,24.208498,3.083145,1.945726,22.137859,0.138564,0.075130,0.005690,2.365910,...,2.337694,0.857740,0.034353,0.576273,0.08504,0.193303,22.781940,234.254422,112.390790,346.645212
std,2.874598,5.000841,8.426519,10.043213,6.407479,73.548204,0.560785,0.366776,0.078651,5.005712,...,0.534881,0.286122,0.046828,0.204585,0.08048,0.130603,4.025229,30.795588,19.561061,41.496193
min,2012.000000,1.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2014.000000,5.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.687500,0.000000,0.437500,0.00000,0.062500,20.187500,221.437500,101.625000,328.125000
50%,2017.000000,9.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.375000,0.812500,0.000000,0.562500,0.06250,0.187500,22.750000,236.187500,112.500000,349.062500
75%,2019.000000,13.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,2.687500,1.062500,0.062500,0.687500,0.12500,0.250000,25.375000,250.750000,123.937500,368.937500
max,2021.000000,17.000000,47.000000,68.000000,45.000000,527.000000,7.000000,6.000000,2.000000,38.000000,...,3.750000,1.937500,0.187500,1.312500,0.37500,0.687500,32.437500,299.750000,165.875000,440.125000


In [166]:
print(len(final))
print(len(final.groupby(['player', 'season'])))

56768
6160


In [167]:
final.loc[final['pos']=='WR']

,season,week,team,player,age,pos,posRank,opponent,passA,passC,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
20290,2012.0,1.0,DAL,Kevin Ogletree,25.0,WR,WR1,NYG,0.0,0.0,...,3.0000,1.2500,0.1250,0.6875,0.0625,0.0625,25.0000,255.1250,121.2500,376.3750
20291,2012.0,1.0,ATL,Julio Jones,23.0,WR,WR1,KCC,0.0,0.0,...,1.8125,1.2500,0.0000,0.3750,0.0000,0.1250,21.1250,201.3125,132.0000,333.3125
20292,2012.0,1.0,NYJ,Stephen Hill,21.0,WR,WR1,BUF,0.0,0.0,...,1.8125,1.2500,0.0625,0.6875,0.1875,0.4375,27.1250,232.1250,139.0000,371.1250
20293,2012.0,1.0,NOS,Lance Moore,29.0,WR,WR1,WAS,0.0,0.0,...,2.5625,0.8125,0.0000,0.5000,0.0625,0.0625,22.9375,222.0625,117.7500,339.8125
20294,2012.0,1.0,HOU,Andre Johnson,31.0,WR,WR1,MIA,0.0,0.0,...,2.5625,1.0000,0.0000,0.1875,0.1250,0.1250,19.5625,249.5000,95.6250,345.1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38603,2021.0,17.0,PHI,Jalen Reagor,22.0,WR,WR4,WAS,0.0,0.0,...,2.9375,1.0000,0.0625,0.4375,0.0000,0.1875,20.5625,191.7500,112.8125,304.5625
38604,2021.0,17.0,GBP,Marquez Valdes-Scantling,27.0,WR,WR5,MIN,0.0,0.0,...,1.4375,0.9375,0.0625,0.4375,0.1250,0.0000,29.6875,258.8125,134.4375,393.2500
38605,2021.0,17.0,MIN,Dede Westbrook,28.0,WR,WR3,GBP,0.0,0.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000
38606,2021.0,17.0,ATL,Olamide Zaccheaus,24.0,WR,WR2,BUF,0.0,0.0,...,2.3750,0.9375,0.0000,0.6875,0.0000,0.2500,23.4375,232.8750,119.6250,352.5000


In [168]:
# Account for the way WRS are set in slot, left, and right
final['posRank'] = final['posRank'].map({
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB3',
    'QB5':'QB3', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB3', 
    'RB5':'RB3',
    'RB6':'RB3', 
    'RB7':'RB3', 
    'WR1': 'WR1', 
    'WR2': 'WR1', 
    'WR3': 'WR1', 
    'WR4': 'WR2', 
    'WR5': 'WR2', 
    'WR6': 'WR2', 
    'WR7': 'WR3', 
    'WR8': 'WR3',
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE3', 
    'TE5':'TE3', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })


In [169]:
len(final)

56768

In [170]:
final.columns

Index(['season', 'week', 'team', 'player', 'age', 'pos', 'posRank', 'opponent',
       'passA', 'passC',
       ...
       'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp',
       'defFum_prior1_opp', 'defBlk_prior1_opp', 'defT_prior1_opp',
       'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp',
       'defRushYAgainst_prior1_opp', 'defYdsAgainst_prior1_opp'],
      dtype='object', length=151)

In [171]:
final.to_csv('data_cleaned/modelSource.csv')

In [ ]:

# create game index to track the sequence of games across seasons
df['gameIndex'] = 'NaN'
counter=0
for i in range(2010, 2022):
    for j in range(1, 18):
        df.loc[(df.season==i) & (df.week==j), 'gameIndex'] = counter
        counter+=1
# Move gameindex column to third position
gameIndexCol = df.pop('gameIndex')
df.insert(2, 'gameIndex', gameIndexCol)

In [ ]:
# find all these stats for the previous three games, current season, previous year, 2 years previous
# find per-game stats by dividing by length
# merge pergame stats
i=0
# for i in range(len(df)):
season = df.iloc[i]['season']
gameIndex = df.iloc[i]['gameIndex']
player = df.iloc[i]['player']

currentSeason = season
seasonsPrior1 = season - 1
seasonsPrior2 = season - 2
weeksPrior3 = gameIndex - 3

df.loc[(df.season==season) & (df.gameIndex>=gameIndex) & (df.player==player), 'weeksPrior3'] = df.loc[(df.season==season) & (df.gameIndex>=gameIndex) & (df.player==player)]['passA'].sum()

priorYear1 = df.loc[(df.season==seasonPrior1) & (df.player==player)]

In [ ]:
# Get ages
